[resource 1](https://github.com/emirhanai/Cryptocurrency-Prediction-with-Artificial-Intelligence/blob/main/Cryptocurrency%20Prediction%20with%20Artificial%20Intelligence.ipynb)

In [ ]:
!pip install alpaca_trade_api #.rest

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [ ]:
import datetime 
history = datetime.timedelta(days = 50)
from datetime import datetime
from datetime import date

# https://pypi.org/project/alpaca-trade-api/
# alpaca api key, alpaca secret key
from alpaca_trade_api.rest import REST, TimeFrame
client = REST('PK1H7PEKFTOLZA5S9HHU', 'NF0RkI5lOHLOS1OXFAlT1DsxIPmgJbHHES2iJr8Z')

trading_pair= 'BTCUSD'
df = client.get_crypto_bars(trading_pair, TimeFrame.Hour, start = date.today() - history, end = date.today()).df

In [ ]:

df = df[df.exchange == 'FTXU']
metric = 'close'
data = df.filter([metric])
data = data.values

In [ ]:
split_perc = .5
training_data_len = int(np.ceil( len(df) * split_perc))
train_data = df.iloc[:training_data_len]
test_data = df.iloc[training_data_len:]

In [ ]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('XRP/USDT', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);
line_plot(train_data[metric], test_data[metric], 'train', 'test', title='')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [ ]:
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 50
epochs = 20
batch_size = 32
loss = 'mse'
dropout = 0.24
optimizer = 'adam'

def LSTM_model(input_data, output_size, neurons, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    # input_shape = (60, 1)
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2]), return_sequences = True))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, return_sequences = True))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model
np.random.seed(245)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
from sklearn.preprocessing import MinMaxScaler
scaled_data = scaler.fit_transform(data)

In [ ]:
train_data = scaled_data[0:int(training_data_len), :]
look_back = 72
x_train = []
y_train = []
for price in range(look_back, len(data)):
    x_train.append(scaled_data[price - look_back:price, :])
    y_train.append(scaled_data[price, :])

In [ ]:

test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = np.array(scaled_data[training_data_len:, :])
for price in range(60, len(test_data)):
  x_test.append(test_data[price - look_back:price, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

model = LSTM_model(
    x_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
# modelfit = model.fit(
#     x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)
modelfit = model.fit(
    x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)


In [ ]:

import matplotlib.pyplot as plt
plt.plot(modelfit.history['loss'],'r',linewidth=2, label='Training loss')
# plt.plot(modelfit.history['val_loss'], 'g',linewidth=2, label='Validation loss')
plt.title('LSTM Neural Networks - XRP Model')
plt.xlabel('Epochs numbers')
plt.ylabel('MSE numbers')
plt.show()

preds = model.predict(x_test).squeeze()
mean_absolute_error(preds, y_test)

from sklearn.metrics import mean_squared_error
SCORE_MSE=mean_squared_error(preds, y_test)
SCORE_MSE

from sklearn.metrics import r2_score
r2_score=r2_score(y_test, preds)
r2_score*100

y_test_true = scaler.inverse_transform(y_test)
preds_true = scaler.inverse_transform(preds)
line_plot(y_test_true, preds_true, 'actual', 'prediction')